In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [93]:
# igraph
!pip install python-igraph
from igraph import *

# package to show plot of igraph
!pip install cairocffi
import cairocffi

In [20]:
import pandas as pd
import numpy as np


In [37]:
# read in distance matrix
industry_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/industry_matrix.csv',index_col=0)
mktcap_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/mktcap_matrix.csv',index_col=0)
n_instruments_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/n_instruments_matrix.csv',index_col=0)
turnover_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/turnover_matrix.csv',index_col=0)
concentration_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/concentration_matrix.csv',index_col=0)
invstyle_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/invStyle_matrix.csv',index_col=0)
total_hld_matrix = pd.read_csv('/content/gdrive/My Drive/capstone/distance matrix/total_hld_matrix.csv',index_col=0)

In [38]:
def get_adjacency_mat_from_dist_mat(dist_mat):
  adjacency_mat = dist_mat.copy()
  np.fill_diagonal(adjacency_mat.values, np.nan)
  max = np.nanmax(adjacency_mat.to_numpy())
  min = np.nanmin(adjacency_mat.to_numpy())

  # Inverse distance to be edge weight
  adjacency_mat = max - adjacency_mat + min
  adjacency_mat.fillna(0, inplace = True)
  
  return adjacency_mat

In [39]:
# convert all 7 dist mat to adj mat
ad_mktcap = get_adjacency_mat_from_dist_mat(mktcap_matrix)
ad_industry = get_adjacency_mat_from_dist_mat(industry_matrix)
ad_turnover = get_adjacency_mat_from_dist_mat(turnover_matrix)
ad_concentration = get_adjacency_mat_from_dist_mat(concentration_matrix)
ad_invstyle = get_adjacency_mat_from_dist_mat(invstyle_matrix)
ad_n_instruments = get_adjacency_mat_from_dist_mat(n_instruments_matrix)
ad_total_hld = get_adjacency_mat_from_dist_mat(total_hld_matrix)

In [68]:
feature_dic = {'portfolio_mkt': ad_mktcap,
               'portfolio_industry': ad_industry,
               'turnover': ad_turnover,
               'concentration': ad_concentration,
               'invStyle': ad_invstyle,
               'n_instruments': ad_n_instruments,
               'total_hld': ad_total_hld}

# normalize all edge weights of each adj mat to [0,1]
for feature in feature_dic:
  feature_ad_mat = feature_dic[feature]
  feature_dic[feature] = feature_ad_mat/feature_ad_mat.max().max()
  feature_dic[feature].index = feature_dic[feature].index.map(str)

In [70]:
# Get the intersection investor list from all 7 adjacency mat
# the investors list is the intersection of all 7 feature mat index
inv_intersect_ls = set(feature_dic['total_hld'].index)
for feature in feature_dic:
  inv_intersect_ls = set(set(feature_dic[feature].index) & inv_intersect_ls)

print(len(inv_intersect_ls))
ls_investors = inv_intersect_ls

225


In [89]:
# # if adjacency matrix have missing investors, fill in
# def fill_missing(df, ls_investors):
#   missing_investors = ls_investors - set(df.index)
#   for index in missing_investors:
#     df.loc[:,str(index)] = 0
#     df.loc[index] = 0
#   df.fillna(0,inplace=True)
#   df.sort_index(inplace=True)
#   return df

# for feature in feature_dic:
#   feature_ad_mat = feature_dic[feature]
#   feature_dic[feature] = fill_missing(feature_ad_mat,ls_investors)


# create feature dict using only investors in the intersection (225) and sort by both col row index
intersect_feature_dic = {}
for feature in feature_dic:  
  adj_mat = feature_dic[feature][ls_investors].loc[ls_investors]
  adj_mat = adj_mat.sort_index(axis = 1).sort_index(axis = 0)
  intersect_feature_dic[feature] = adj_mat


# create an empty adjacency matrix using sorted intersection index
adjacency = pd.DataFrame(data = 0, 
                         index=intersect_feature_dic['total_hld'].index, 
                         columns=intersect_feature_dic['total_hld'].index)


# add up all features
for feature in intersect_feature_dic:
  adjacency += intersect_feature_dic[feature]

# # change column order to make column order coordinate with index order
# adjacency = adjacency[[str(i) for i in list(adjacency.index)]]

In [90]:
adjacency

LGCYINVESTORID,10061989,10075382,10110259,10233291,10235680,2000264,2001934,2001935,2001989,2001999,2002019,2002142,2002146,2002158,2002179,2002180,2002195,2002205,2002222,2002292,2002295,2002303,2002362,2002389,2002446,2002451,2002468,2002496,2002506,2002511,2002548,2002582,2002628,2002658,2002659,2002678,2002695,2002713,2002715,2002801,...,3015075,3318171,3365036,3676543,3706224,3828352,3965008,4040011,4059340,4889409,4924814,4950170,4962258,5050280,5200477,5205742,5206490,5206923,5275446,5308615,5436717,5480445,5505145,5520904,5529187,5713469,5819294,5844860,5915225,5986931,6053249,6129655,6139212,6248967,6271047,6921752,7185253,9098246,9113038,9969058
LGCYINVESTORID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10061989,0.000000,5.117883,4.443423,5.230592,5.047739,4.842475,5.023472,4.740072,5.066365,4.926741,5.643603,5.276924,5.381739,5.096421,4.803101,5.029735,5.050604,5.075438,4.874387,5.596114,5.004836,5.538905,5.076159,5.415548,5.196345,4.780825,4.889076,5.413981,4.966220,4.667987,5.109988,4.920678,5.489124,4.819141,5.203088,5.123322,5.353369,4.731662,4.617665,5.457465,...,5.121113,4.856562,4.806564,5.371328,5.226285,5.227323,5.356751,5.295001,4.694719,4.601240,4.908843,5.138862,4.990374,5.283594,4.254359,5.148458,5.421754,5.332191,5.558167,4.485573,4.707733,5.558091,4.563647,4.979926,5.158062,5.916761,5.044509,5.359866,5.221795,4.414188,5.524020,5.782314,5.337294,5.272897,5.270389,4.612794,5.046228,5.199865,4.406134,5.539406
10075382,5.117883,0.000000,5.597658,5.913743,5.890947,4.840502,4.187476,3.970648,4.458919,4.150608,5.078726,5.095742,4.836948,4.358852,4.754323,4.647800,4.260273,4.355428,4.543296,5.098704,4.475552,5.224117,4.441618,5.029265,4.546818,3.957892,4.160610,4.781714,4.246561,3.922354,4.328697,4.114551,4.841495,4.236802,4.567258,4.396654,4.624315,4.069299,3.934612,4.677277,...,4.673919,4.508852,4.112984,5.339365,5.196943,5.986658,5.479327,5.513830,4.014410,4.086181,4.722353,4.775532,5.513655,5.241732,4.908497,4.956269,5.120643,4.960155,5.140098,4.551455,4.161327,4.967918,3.886590,4.573093,4.695447,5.141205,5.843342,5.088918,4.655650,4.480968,4.743729,5.100865,5.469612,4.526607,4.770405,4.294837,4.302241,5.185021,4.696609,5.653267
10110259,4.443423,5.597658,0.000000,5.377915,5.241696,5.347374,4.697801,3.700966,5.151012,4.587845,4.406914,5.340854,4.986874,4.915128,5.215037,5.277254,4.673428,4.942300,4.909382,4.821246,4.682006,5.072880,5.068446,4.236903,4.943798,4.547374,4.257130,4.910916,4.392177,4.508118,4.795527,4.774177,4.874549,4.556678,5.070390,4.886238,4.801988,4.469910,4.549807,4.898168,...,4.541693,4.268475,4.596269,4.721506,4.383409,5.549791,4.747518,4.837884,4.656162,4.729560,4.681633,4.931748,5.163454,4.957441,5.620728,4.890949,4.545987,4.962929,4.435229,4.108715,4.732730,4.753704,4.499867,3.781667,4.818501,4.529664,5.459207,4.385047,4.830760,4.745572,4.112105,4.430296,4.728527,5.134461,4.136295,4.955252,4.991951,4.495316,5.135658,5.012965
10233291,5.230592,5.913743,5.377915,0.000000,6.456974,4.767015,4.202148,4.085740,4.686595,4.215563,5.287623,5.166357,5.033237,4.468913,4.963340,4.714202,4.307930,4.477341,4.645422,5.011951,4.222421,5.123549,4.588318,4.757883,4.438351,4.019862,4.298667,4.754536,4.384560,3.821824,4.416161,4.188990,4.904006,4.441571,4.704210,4.456751,4.705694,4.196872,3.767585,4.682769,...,4.881713,4.267859,4.015103,5.634658,4.934530,6.180806,5.356810,5.760839,4.034098,4.489333,4.902112,4.575807,6.304697,5.061020,5.254205,4.995914,4.983765,4.944095,5.236454,4.549170,4.384916,4.831768,3.995697,4.363835,4.473830,5.486064,5.886350,5.018020,4.619410,4.615082,4.975116,5.132989,5.465780,4.581003,4.823354,4.266616,4.357847,5.193580,4.617180,5.835204
10235680,5.047739,5.890947,5.241696,6.456974,0.000000,4.710823,4.089859,3.922397,4.459292,4.075168,5.124102,5.005697,4.834408,4.363212,4.804657,4.576775,4.172212,4.360113,4.472499,4.889200,4.135842,4.977657,4.372361,4.662159,4.405169,3.893846,4.131323,4.652492,4.217214,3.727469,4.292554,4.032409,4.770081,4.

In [114]:
def drop_edges_output_mat(adjacency, pct=0.5, thres = None):
  '''
  Remove some proportion of edges if the weight is low. If threshold is provided, use the threshold
  '''
  if thres is None:
    thres = np.percentile(adjacency.to_numpy(), pct*100)
  print('threshold is', thres)
  # remove bottom pct of edges
  adjacency_filtered = adjacency[adjacency > thres].fillna(0)
  return adjacency_filtered

# drop_edges_output_mat(adjacency, thres = 6).to_csv('/content/gdrive/My Drive/capstone/distance matrix/all_features_adj_mat_thres_6.csv')

threshold is 6


In [108]:
len(set(drop_edges_output_mat(adjacency, thres = 5).columns))

threshold is 5


225

In [91]:
def drop_edges(adjacency, pct=0.5, thres = None):
  '''
  Remove some proportion of edges if the weight is low. If threshold is provided, use the threshold
  '''
  if thres is None:
    # only compute the threshold if threshold is not provided
    thres = np.percentile(adjacency.to_numpy(), pct*100)
  print('threshold is', thres)
  # remove bottom pct of edges
  adjacency_filtered = adjacency[adjacency > thres].fillna(0)
  # print(adjacency_filtered)
  # create undirected, weighted graph
  g = Graph.Weighted_Adjacency(adjacency_filtered.values.tolist(),mode=ADJ_UNDIRECTED)
  return g

In [100]:

g = drop_edges(adjacency,thres=2)
plot(g.community_walktrap(weights=g.es["weight"]).as_clustering(),
     vertex_label = ls_investors,
     vertex_size = 16)


threshold is 2


TypeError: ignored

In [ ]:
# # construct a layout
# layout = g.layout_fruchterman_reingold(weights=g.es["weight"])

# # construct the plot settings
# plot_settings = dict(
#         layout=layout,
#         edge_width=rescale(g.es["weight"], out_range=(0.0, 5.0))
# )

# # plot the graph
# plot(g.community_walktrap(weights=g.es["weight"]).as_clustering(),**plot_settings)
# # plot(g, **plot_settings)